In [53]:
# --- 1. Cài đặt các thư viện cần thiết ---
!pip install streamlit plotly pandas seaborn pyngrok -q
!pip install pyngrok --upgrade -q

In [54]:
# --- 2. Tạo file app.py ---
# Nội dung file: app.py
app_code = '''
import streamlit as st
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import time

# ====== CẤU HÌNH GIAO DIỆN TỐI ======
st.set_page_config(
    page_title="AirVision Analytics",
    page_icon="🌍",
    layout="wide",
    initial_sidebar_state="expanded"
)

# Tùy chỉnh CSS cho giao diện tối
def local_css(file_name):
    with open(file_name) as f:
        st.markdown(f'<style>{f.read()}</style>', unsafe_allow_html=True)

# Tạo file CSS tạm với theme tối
custom_css = """
:root {
    --primary-color: #4a8fe7;
    --secondary-color: #2c3e50;
    --accent-color: #ff6b6b;
    --dark-color: #1a1a1a;
    --light-color: #f8f9fa;
}

/* Main container */
.stApp {
    background-color: var(--dark-color);
    color: #ffffff;
}

/* Header */
header {
    background: linear-gradient(135deg, var(--primary-color), #3a7bd5);
    color: white !important;
    padding: 1rem 2rem !important;
    border-radius: 0 0 10px 10px;
    box-shadow: 0 4px 12px rgba(0,0,0,0.3);
}

/* Sidebar */
[data-testid="stSidebar"] {
    background-color: var(--secondary-color) !important;
    border-right: 1px solid #444;
    box-shadow: 2px 0 10px rgba(0,0,0,0.3);
    color: white;
}

/* Cards */
.custom-card {
    background: var(--secondary-color);
    color: white;
    border-radius: 10px;
    padding: 1.5rem;
    box-shadow: 0 4px 12px rgba(0,0,0,0.2);
    margin-bottom: 1.5rem;
    border-left: 4px solid var(--primary-color);
}

/* Buttons */
.stButton>button {
    border-radius: 8px !important;
    border: none !important;
    background-color: var(--primary-color) !important;
    color: white !important;
    transition: all 0.3s ease !important;
}

.stButton>button:hover {
    transform: translateY(-2px);
    box-shadow: 0 4px 8px rgba(0,0,0,0.3) !important;
}

/* Tabs */
[data-baseweb="tab-list"] {
    gap: 10px;
}

[data-baseweb="tab"] {
    padding: 8px 16px !important;
    border-radius: 8px !important;
    background-color: #34495e !important;
    color: white !important;
    transition: all 0.3s ease !important;
}

[data-baseweb="tab"]:hover {
    background-color: #2c3e50 !important;
}

[aria-selected="true"] {
    background-color: var(--primary-color) !important;
    color: white !important;
}

/* Progress bar */
.stProgress > div > div > div > div {
    background-color: var(--primary-color) !important;
}

/* Tables */
.stDataFrame {
    background-color: var(--secondary-color) !important;
    color: white !important;
}
"""

with open('styles.css', 'w') as f:
    f.write(custom_css)
local_css('styles.css')

# ====== ANIMATION & EFFECTS ======
def render_animated_header():
    st.markdown("""
    <div style="display: flex; align-items: center; justify-content: space-between;">
        <div>
            <h1 style="color: white; margin-bottom: 0;">🌍 AirVision Analytics</h1>
            <p style="color: white; opacity: 0.9; margin-top: 0.5rem;">Trực quan hóa dữ liệu chất lượng không khí & khí tượng</p>
        </div>
        <div style="background: rgba(255,255,255,0.2); padding: 0.5rem 1rem; border-radius: 20px; display: flex; align-items: center;">
            <svg width="24" height="24" viewBox="0 0 24 24" fill="none" xmlns="http://www.w3.org/2000/svg" style="margin-right: 0.5rem;">
                <path d="M12 2C6.48 2 2 6.48 2 12C2 17.52 6.48 22 12 22C17.52 22 22 17.52 22 12C22 6.48 17.52 2 12 2ZM12 20C7.59 20 4 16.41 4 12C4 7.59 7.59 4 12 4C16.41 4 20 7.59 20 12C20 16.41 16.41 20 12 20Z" fill="white"/>
                <path d="M12 6C8.69 6 6 8.69 6 12C6 15.31 8.69 18 12 18C15.31 18 18 15.31 18 12C18 8.69 15.31 6 12 6ZM12 16C9.79 16 8 14.21 8 12C8 9.79 9.79 8 12 8C14.21 8 16 9.79 16 12C16 14.21 14.21 16 12 16Z" fill="white"/>
                <path d="M12 10C10.9 10 10 10.9 10 12C10 13.1 10.9 14 12 14C13.1 14 14 13.1 14 12C14 10.9 13.1 10 12 10Z" fill="white"/>
            </svg>
            <span style="color: white;">Real-time Monitoring</span>
        </div>
    </div>
    """, unsafe_allow_html=True)

# ====== HÀM TẢI DỮ LIỆU ======
@st.cache_data
def load_data():
    url = "https://redcap.huph.edu.vn/ddp/tsa/nhom4.csv"
    try:
        # Đọc file CSV từ dữ liệu đã upload
        df = pd.read_csv(url, sep=',', decimal='.')

        # Kết hợp cột Date và Time thành Datetime
        df['Datetime'] = pd.to_datetime(
            df['Date'] + ' ' + df['Time'],
            format='%m/%d/%Y %H:%M:%S',
            errors='coerce'
        )

        # Xóa các dòng có Datetime không hợp lệ
        df = df.dropna(subset=['Datetime'])

        # Sắp xếp theo thời gian
        df = df.sort_values('Datetime')

        # Thay thế giá trị -200 bằng NaN
        df = df.replace(-200, np.nan)

        return df

    except Exception as e:
        st.error(f"Lỗi khi đọc dữ liệu: {str(e)}")
        return None


# ====== MAIN APP ======
def main():
    render_animated_header()

    # Tạo hiệu ứng loading
    with st.spinner('Đang tải dữ liệu và khởi tạo ứng dụng...'):
        time.sleep(1)

    # Tải dữ liệu
    df = load_data()

    if df is None or df.empty:
        st.error("Không thể tải dữ liệu hoặc dữ liệu trống. Vui lòng kiểm tra kết nối hoặc định dạng dữ liệu.")
        return

    # Kiểm tra các cột có trong dữ liệu
    available_columns = df.columns.tolist()
    pollutants = ['CO(GT)', 'NOx(GT)', 'PT08.S5(O3)', 'C6H6(GT)']
    weather = ['T', 'RH', 'AH']

    # ====== SIDEBAR HIỆN ĐẠI ======
    with st.sidebar:
        st.markdown("""
        <div style="display: flex; align-items: center; margin-bottom: 1.5rem;">
            <svg width="24" height="24" viewBox="0 0 24 24" fill="none" xmlns="http://www.w3.org/2000/svg" style="margin-right: 0.5rem;">
                <path d="M10 20V14H14V20H19V12H22L12 3L2 12H5V20H10Z" fill="#4a8fe7"/>
            </svg>
            <h2 style="margin: 0; color: white;">Bộ lọc dữ liệu</h2>
        </div>
        """, unsafe_allow_html=True)

        # Date/Time Filter
        with st.expander("🗓️ Lọc theo thời gian", expanded=True):
            min_date = df['Datetime'].min().date()
            max_date = df['Datetime'].max().date()

            col1, col2 = st.columns(2)
            with col1:
                start_date = st.date_input("Từ ngày", min_date, key="start_date")
            with col2:
                end_date = st.date_input("Đến ngày", max_date, key="end_date")

            col3, col4 = st.columns(2)
            with col3:
                start_time = st.time_input("Từ giờ", datetime.strptime("00:00:00", "%H:%M:%S").time(), key="start_time")
            with col4:
                end_time = st.time_input("Đến giờ", datetime.strptime("23:59:59", "%H:%M:%S").time(), key="end_time")

        # Biến lựa chọn
        with st.expander("📊 Chọn biến hiển thị", expanded=True):
            st.markdown("**Chất ô nhiễm**", help="Chọn các chất ô nhiễm để hiển thị")
            selected_pollutants = []
            for poll in pollutants:
                if poll in available_columns:
                    selected_pollutants.append(st.checkbox(poll, True, key=f"poll_{poll}"))
                else:
                    st.warning(f"Không tìm thấy cột {poll} trong dữ liệu")

            st.markdown("**Thông số khí tượng**", help="Chọn các thông số khí tượng để hiển thị")
            selected_weather = []
            for w in weather:
                if w in available_columns:
                    selected_weather.append(st.checkbox(w, True, key=f"weather_{w}"))
                else:
                    st.warning(f"Không tìm thấy cột {w} trong dữ liệu")

    # ====== MAIN DASHBOARD ======
    # Lọc dữ liệu theo thời gian
    try:
        start_dt = datetime.combine(start_date, start_time)
        end_dt = datetime.combine(end_date, end_time)
        df_filtered = df[(df['Datetime'] >= start_dt) & (df['Datetime'] <= end_dt)]
    except Exception as e:
        st.error(f"Lỗi khi lọc dữ liệu theo thời gian: {str(e)}")
        df_filtered = df.copy()

    # Tạo tabs
    tab1, tab2, tab3 = st.tabs(["📈 Dashboard", "🔍 Phân tích", "📤 Dữ liệu"])

    with tab1:
        # Thống kê nhanh
        st.markdown("### 📊 Tổng quan dữ liệu")
        col1, col2, col3 = st.columns(3)
        with col1:
            st.markdown(f"""
            <div class="custom-card">
                <h3 style="color: var(--primary-color); margin-top: 0;">Thời gian</h3>
                <p>Bắt đầu: {df_filtered['Datetime'].min().strftime('%d/%m/%Y %H:%M')}</p>
                <p>Kết thúc: {df_filtered['Datetime'].max().strftime('%d/%m/%Y %H:%M')}</p>
            </div>
            """, unsafe_allow_html=True)

        with col2:
            st.markdown(f"""
            <div class="custom-card">
                <h3 style="color: var(--primary-color); margin-top: 0;">Số lượng dữ liệu</h3>
                <p>Tổng số mẫu: {len(df_filtered):,}</p>
                <p>Số ngày: {(df_filtered['Datetime'].max() - df_filtered['Datetime'].min()).days + 1}</p>
            </div>
            """, unsafe_allow_html=True)

        with col3:
            avg_co = df_filtered['CO(GT)'].mean() if 'CO(GT)' in df_filtered.columns else 'N/A'
            avg_nox = df_filtered['NOx(GT)'].mean() if 'NOx(GT)' in df_filtered.columns else 'N/A'
            st.markdown(f"""
            <div class="custom-card">
                <h3 style="color: var(--primary-color); margin-top: 0;">Chất lượng không khí</h3>
                <p>CO trung bình: {avg_co if isinstance(avg_co, str) else f"{avg_co:.2f} ppm"}</p>
                <p>NOx trung bình: {avg_nox if isinstance(avg_nox, str) else f"{avg_nox:.2f} ppb"}</p>
            </div>
            """, unsafe_allow_html=True)

        # Biểu đồ chính
        st.markdown("### 🌡️ Biểu đồ theo dõi")
        if not df_filtered.empty:
            fig = make_subplots(specs=[[{"secondary_y": True}]])

            # Thêm dữ liệu chất ô nhiễm
            for poll in [p for p in pollutants if p in df_filtered.columns]:
                fig.add_trace(
                    go.Scatter(
                        x=df_filtered['Datetime'],
                        y=df_filtered[poll],
                        name=poll,
                        line=dict(width=2),
                        mode='lines',
                        hovertemplate="%{y:.2f}<extra></extra>"
                    ),
                    secondary_y=False
                )

            # Thêm dữ liệu khí tượng
            for w in [w for w in weather if w in df_filtered.columns]:
                fig.add_trace(
                    go.Scatter(
                        x=df_filtered['Datetime'],
                        y=df_filtered[w],
                        name=w,
                        line=dict(dash='dot', width=1.5),
                        mode='lines',
                        hovertemplate="%{y:.2f}<extra></extra>"
                    ),
                    secondary_y=True
                )

            fig.update_layout(
                height=500,
                template='plotly_dark',
                hovermode="x unified",
                legend=dict(
                    orientation="h",
                    yanchor="bottom",
                    y=1.02,
                    xanchor="right",
                    x=1
                ),
                margin=dict(l=20, r=20, t=40, b=20),
                plot_bgcolor='rgba(0,0,0,0)',
                paper_bgcolor='rgba(0,0,0,0)'
            )

            fig.update_yaxes(title_text="Nồng độ chất ô nhiễm", secondary_y=False)
            fig.update_yaxes(title_text="Thông số khí tượng", secondary_y=True)

            st.plotly_chart(fig, use_container_width=True)

    with tab2:
        st.markdown("### 🔥 Phân tích tương quan")
        # Lọc các cột có trong dữ liệu
        available_vars = [v for v in pollutants + weather if v in df_filtered.columns]

        if len(available_vars) >= 2:
            corr_matrix = df_filtered[available_vars].corr()

            fig, ax = plt.subplots(figsize=(10, 8))
            sns.heatmap(
                corr_matrix,
                annot=True,
                fmt=".2f",
                cmap="coolwarm",
                center=0,
                square=True,
                linewidths=.5,
                cbar_kws={"shrink": .8},
                ax=ax
            )
            plt.title("Ma trận tương quan giữa các thông số", pad=20, color='white')
            ax.set_facecolor('#2c3e50')
            fig.patch.set_facecolor('#2c3e50')
            ax.tick_params(axis='x', colors='white')
            ax.tick_params(axis='y', colors='white')
            st.pyplot(fig)

            # Phân tích xu hướng
            st.markdown("### 📉 Phân tích xu hướng")
            selected_trend = st.selectbox("Chọn thông số để phân tích xu hướng", available_vars)

            if selected_trend in df_filtered.columns:
                trend_fig = go.Figure()

                trend_fig.add_trace(
                    go.Scatter(
                        x=df_filtered['Datetime'],
                        y=df_filtered[selected_trend],
                        name='Giá trị thực',
                        line=dict(color='#4a8fe7')
                    )
                )

                # Thêm đường trung bình 7 ngày
                rolling_mean = df_filtered.set_index('Datetime')[selected_trend].rolling('7D').mean()
                trend_fig.add_trace(
                    go.Scatter(
                        x=rolling_mean.index,
                        y=rolling_mean.values,
                        name='Trung bình 7 ngày',
                        line=dict(color='#ff6b6b', dash='dash')
                    )
                )

                trend_fig.update_layout(
                    title=f"Xu hướng {selected_trend}",
                    xaxis_title="Thời gian",
                    yaxis_title="Giá trị",
                    template='plotly_dark',
                    plot_bgcolor='rgba(0,0,0,0)',
                    paper_bgcolor='rgba(0,0,0,0)'
                )

                st.plotly_chart(trend_fig, use_container_width=True)

    with tab3:
        st.markdown("### 📂 Dữ liệu thô")

        # Hiển thị toàn bộ dữ liệu dạng bảng
        st.dataframe(
            df_filtered,
            height=600,  # Chiều cao cố định
            use_container_width=True,
            hide_index=True  # Ẩn chỉ số dòng
        )

        # Thông tin tóm tắt trong expander
        with st.expander("ℹ️ Thông tin dữ liệu"):
            cols = st.columns(3)
            with cols[0]:
                st.metric("Số dòng", len(df_filtered))
            with cols[1]:
                st.metric("Số cột", len(df_filtered.columns))
            with cols[2]:
                st.metric("Dữ liệu thiếu", df_filtered.isna().sum().sum())

        # Tùy chọn xuất dữ liệu
        st.markdown("### 📤 Xuất dữ liệu")
        export_format = st.radio("Định dạng xuất", ["CSV", "Excel"], horizontal=True)

        if st.button("Xuất dữ liệu đã lọc", type="primary"):
            if export_format == "CSV":
                csv = df_filtered.to_csv(index=False).encode('utf-8')
                st.download_button(
                    label="Tải xuống CSV",
                    data=csv,
                    file_name=f"air_quality_{start_date}_to_{end_date}.csv",
                    mime="text/csv"
                )
            else:
                from io import BytesIO
                output = BytesIO()
                with pd.ExcelWriter(output, engine='xlsxwriter') as writer:
                    df_filtered.to_excel(writer, index=False)
                excel_data = output.getvalue()
                st.download_button(
                    label="Tải xuống Excel",
                    data=excel_data,
                    file_name=f"air_quality_{start_date}_to_{end_date}.xlsx",
                    mime="application/vnd.ms-excel"
                )

if __name__ == "__main__":
    main()
'''

with open('/content/app.py', 'w') as f:
    f.write(app_code)

In [55]:
# --- 3. Khởi chạy ứng dụng ---
from pyngrok import ngrok
import subprocess
import threading
import time
import requests

# Reset ngrok tunnels
ngrok.kill()

# Cấu hình Ngrok
NGROK_TOKEN = "2wIdHLEeJI9sBLsjQ3Lkt30ZYkT_33pMAYJokqqqyypmD9Zz9"  # 👈 Thay token của bạn
ngrok.set_auth_token(NGROK_TOKEN)

# Khởi chạy Streamlit
def run_streamlit():
    subprocess.run([
        "streamlit", "run",
        "/content/app.py",
        "--server.port", "8501",
        "--server.headless", "true",
        "--browser.gatherUsageStats", "false"
    ])

threading.Thread(target=run_streamlit, daemon=True).start()

# Chờ khởi động
time.sleep(8)

# Tạo public URL
try:
    tunnel = ngrok.connect(8501, "http")
    public_url = tunnel.public_url
    print(f"\n🔗 Truy cập ứng dụng tại: {public_url}")
    print(f"📊 Xem dashboard Ngrok: https://dashboard.ngrok.com/status/tunnels")

except Exception as e:
    print(f"\n❌ Lỗi kết nối: {str(e)}")
    print("👉 Nguyên nhân thường gặp:")
    print("- Hết hạn token Ngrok")
    print("- Chạy quá 3 tunnel cùng lúc (tài khoản miễn phí)")
    print("- Lỗi mạng")



🔗 Truy cập ứng dụng tại: https://d15c-34-139-5-210.ngrok-free.app
📊 Xem dashboard Ngrok: https://dashboard.ngrok.com/status/tunnels
